In [1]:
import tensorflow as tf
import numpy as np

import keras
import pickle
import h5py
import time
import gzip

from tensorflow.python.ops import data_flow_ops
from tensorflow.keras.layers import Embedding

Using TensorFlow backend.


### Load the input vectors

In [0]:
with gzip.open('./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/train.chunk.pickle', 'rb') as f:
    traindata = pickle.load(f)

train_product   = traindata['product'][:]
train_wproduct = traindata['w_product'][:]
train_label   = traindata['label'][:]

with gzip.open('./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/valid.chunk.pickle', 'rb') as f:
    validdata = pickle.load(f)

valid_product   = validdata['product'][:]
valid_wproduct  = validdata['w_product'][:]
valid_label   = validdata['label'][:]

In [0]:
batch_size = 512
embd_size  = 1024
voca_size  = 100001
lr         = 1e-4
n_epochs   = 3
valid_freq = 3
max_len    = np.shape(train_product)[1]
n_classes  = np.max(train_label) + 1

### Placeholders

In [0]:
products   = tf.placeholder(dtype=tf.float32, shape=[None, max_len], name='products')
w_products = tf.placeholder(dtype=tf.float32, shape=[None, max_len], name='weight_products')
labels   = tf.placeholder(dtype=tf.int32, shape=[None], name='labels')
is_train = tf.placeholder(dtype=tf.bool, name='is_train')

### Deep neural network

In [0]:
# Define embedding
embd = Embedding(voca_size, embd_size, name='product_embd')                     # Define embedding function

# Product embedding
t_product_embd = embd(products)                                                 # Get the embedding vector of products

# Multiply the weights of product vectors
w_product_mat = tf.reshape(w_products, [-1, max_len, 1])
product_embd_mat = tf.keras.layers.dot([t_product_embd, w_product_mat], axes=1) # Multiply the counts of products to the corresponding embedding vectors
product_embd = tf.reshape(product_embd_mat, [-1, embd_size])

# Non-linear activations
embd_out = tf.layers.dropout(product_embd, training=is_train)                   # Dropout
bn = tf.layers.batch_normalization(embd_out, training=is_train)                 # Batch normalization
relu = tf.nn.relu(bn)                                                           # ReLU function

# logits
logits = tf.layers.dense(relu, n_classes)                                       # Fully-connected layer

# Softmax
preds = tf.nn.softmax(logits, name='predictions')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.Dense instead.


### Loss funciton & Optimizer

In [0]:
# Softmax cross entropy loss
loss = tf.losses.softmax_cross_entropy(onehot_labels=tf.one_hot(labels, n_classes), logits=logits)

# Weight decay
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([loss] + reg_losses, name='total_loss')

# Optimizer
optm = tf.train.AdamOptimizer(lr)
train_op = optm.minimize(loss, global_step=tf.train.get_global_step(), name='step_update')
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
train_op = tf.group([train_op, update_ops])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Accuracy

In [0]:
top1_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=1)
top1_acc = tf.identity(top1_acc, name='top1_acc')

top5_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=5)
top5_acc = tf.identity(top5_acc, name='top5_acc')

### Batch generator

In [0]:
def generator(mode='training'):
    if mode == 'training':
        n_data = len(train_label)
        indices = np.arange(n_data)
        np.random.shuffle(indices)
        
        for start_idx in range(0, n_data, batch_size):
            if start_idx + batch_size <= n_data:
                excerpt = indices[start_idx: start_idx + batch_size]
                yield train_product[excerpt, :], train_wproduct[excerpt, :], train_label[excerpt]

    elif mode == 'valid':
        n_data = len(valid_label)
        indices = np.arange(n_data)

        for start_idx in range(0, n_data, batch_size):
            if start_idx + batch_size <= n_data:
                excerpt = indices[start_idx: start_idx + batch_size]
                yield valid_product[excerpt, :], valid_wproduct[excerpt, :], valid_label[excerpt]

### Training session

In [0]:
with tf.Session() as sess:
    tic = time.time()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        print("\n\nEpoch {0:03d} / {1:03d}\n".format(epoch, n_epochs))
        training_loss = []
        for b_product, bw_product, b_label in generator(mode='training'):
            feed_dict = {products  : b_product,
                         w_products: bw_product,
                         labels    : b_label,
                         is_train  : True}
            _, train_loss = sess.run([train_op, loss], feed_dict=feed_dict)
            training_loss.append(train_loss)

        toc = time.time()
        print("[*] TRAIN Loss {0:.4f} | Time {1:.2f}s".format(np.mean(training_loss), toc - tic))
        
        if (epoch+1) % valid_freq == 0:
            TOP1, TOP5 = [], []
            for b_product, bw_product, b_label in generator(mode='valid'):
                feed_dict = {products : b_product,
                            w_products: bw_product,
                            labels    : b_label,
                            is_train  : False}
                t1_acc, t5_acc = sess.run([top1_acc, top5_acc], feed_dict=feed_dict)
                TOP1.append(t1_acc)
                TOP5.append(t5_acc)
            print("[*] VALIDATION Top-1 Acc: {0:.4f} | Top-5 Acc: {1:.4f}".format(np.mean(TOP1), np.mean(TOP5)))

    saver.save(sess, './drive/My Drive/Colab Notebooks/CoE202_KakaoArena/models/dnn_models')



Epoch 000 / 003

[*] TRAIN Loss 4.5290 | Time 50.23s


Epoch 001 / 003

[*] TRAIN Loss 2.0962 | Time 97.14s


Epoch 002 / 003

[*] TRAIN Loss 1.2322 | Time 144.04s
[*] VALIDATION Top-1 Acc: 0.7439 | Top-5 Acc: 0.9065


## Test process
### Load the test data

In [0]:
# test data
with gzip.open('./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/test.chunk.pickle', 'rb') as f:
    testdata = pickle.load(f)

test_product  = testdata['product'][:]
test_wproduct = testdata['w_product'][:]
pids          = testdata['pids'][:]

### Batch generator for test dataset

In [0]:
def generator(mode='test'):
    if mode == 'test':
        n_data = len(test_product)
        indices = np.arange(n_data)
        
        for start_idx in range(0, n_data, batch_size):
            excerpt = indices[start_idx: start_idx + batch_size]
            yield test_product[excerpt, :], test_wproduct[excerpt, :]

In [0]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/models/dnn_models.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/models'))
    DNN = tf.get_default_graph()

    preds = []
    for b_product, bw_product in generator(mode='test'):
        feed_dict = {DNN.get_tensor_by_name('products:0')       : b_product,
                     DNN.get_tensor_by_name('weight_products:0'): bw_product,
                     DNN.get_tensor_by_name('is_train:0')     : False}

        pred = sess.run(DNN.get_tensor_by_name('predictions:0'), feed_dict=feed_dict)
        preds.extend(pred)


INFO:tensorflow:Restoring parameters from ./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/models/dnn_models


### Save the submission files

In [0]:
# Indexing of predictions
argpreds = np.argmax(preds, axis=1)

# Load label dictionary
with open('./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/y_vocab.pickle', 'rb') as f:
    y_dict = pickle.load(f)
# y_dict = pickle.loads(open('./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/y_vocab.pickle').read())

# Inverse label dictionary
inv_y_dict = dict((y,x) for x,y in y_dict.items())
submissions = [inv_y_dict[argpred] for argpred in argpreds]

# Write the results to 'submissions.csv'
f = open('./drive/My Drive/Colab Notebooks/CoE202_KakaoArena/submissions.csv', 'w')
for i, j in zip(pids, submissions):
    line = '{},{}\n'.format(i,j)
    f.write(line)
f.close()

You should submit the 'submissions.csv' file and 4 tf.save files ('checkpoint', 'dnn_models.data', 'dnn_models.index', 'dnn_models.meta') in models folder